<a href="https://colab.research.google.com/github/suchitbhayani/Psychtweets/blob/main/model_training/final_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We will use a RNN this time and see if that helps.

https://www.youtube.com/watch?v=AsNTP8Kwu80

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertModel, get_scheduler
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch.optim import AdamW

In [2]:
from google.colab import files
uploaded = files.upload()

Saving cleaned.csv to cleaned.csv


# Further Preprocessing

In [3]:
data = pd.read_csv('/content/cleaned.csv').drop(columns=['Unnamed: 0'])
data

,cleaned text,label
0,"the pope is infallible, this is a catholic dog...",intj
1,"being you makes you look cute on, because then...",intj
2,"i'm like entp but idiotichey boy, do you want ...",intj
3,give it to ... he has pica since childhood say...,intj
4,frances farmer will have her revenge on seattl...,intj
...,...,...
7232,"god,,pls take care hiro emergency room???? are...",intp
7233,wow last time i got intp i think u upset the f...,intp
7234,a 100% that someone will get his ass kicked so...,entp
7235,if you’re #intj this one is for you | what is ...,infj


In [4]:
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])
data

,cleaned text,label
0,"the pope is infallible, this is a catholic dog...",10
1,"being you makes you look cute on, because then...",10
2,"i'm like entp but idiotichey boy, do you want ...",10
3,give it to ... he has pica since childhood say...,10
4,frances farmer will have her revenge on seattl...,10
...,...,...
7232,"god,,pls take care hiro emergency room???? are...",11
7233,wow last time i got intp i think u upset the f...,11
7234,a 100% that someone will get his ass kicked so...,3
7235,if you’re #intj this one is for you | what is ...,8


# Dataset Class

In [5]:
class MBTIDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    # returns a sample from dataset based on given idx
    def __getitem__(self, idx):
        item_dct = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item_dct['labels'] = torch.tensor(self.labels[idx])
        return item_dct

In [6]:
train_text, test_text, train_labels, test_labels = train_test_split(data['cleaned text'].tolist(), data['label'].tolist(), test_size=0.2, random_state=45)

In [7]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(text=train_text, truncation=True, padding=True)
test_encodings = tokenizer(text=test_text, truncation=True, padding=True)

train_data = MBTIDataset(train_encodings, train_labels)
test_data = MBTIDataset(test_encodings, test_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
train_data[0]

{'input_ids': tensor([  101,  7098,  1997,  2017,  2035,  1045,  3984, 12399,  2063,  2038,
          2025,  2042, 17060,  3689,  2005,  1037,  3232,  1997,  2420,  1010,
          2024,  2017,  4364,  3110,  7929,  1029,  7929, 22708,   999,  2054,
          2006,  3011,  2052,  2191,  2017,  2228,  2008,  1012,  1012,  1012,
          1052,  4246,  2102,  1012,  1012,  1012,  2065,  1045,  2123,  1521,
          1056,  5256,  2039,  2000,  2070,  3492, 15281,  1999,  2026,  2606,
          2059,  1045,  8415,  2000,  2643,  1012,  1012,  1012,  2065,  2017,
          2123,  1521,  1056,  3191,  2026, 12385,  2094,  2077,  2206,  2033,
          2008,  1521,  1055,  1037,  2017,  3291,  1012,  2298,  2012,  2017,
         11065,  2026,  4485,  2525,  1010,  2057,  1521,  2128,  5306,  2085,
         22747,  2860, 18411,  2015,  4429,  4658,  1010, 24978,  2546,  2860,
          2025,  4658,  3597,  4747,  1051,  5358,  2546,  2904,  2000,  1037,
         24978,  2546,  2860,  1006,  1

# Building the Neural Network

https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

https://www.youtube.com/watch?v=ZTt9gsGcdDo&list=PLblh5JKOoLUIxGDQs4LFFD--41Vzf-ME1&index=23

In [9]:
class MBTIPredictor(nn.Module):
    def __init__(self, hidden_layer_size, sequence_length):
        super(MBTIPredictor, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.rnn = nn.RNN(768, hidden_layer_size, batch_first=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(32768, 16)
        # confused hidden_layer_size * sequence_length (= 32000) doesn't work instead of 32768

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        rnn_out, _ = self.rnn(outputs.last_hidden_state)
        rnn_out = rnn_out.reshape(rnn_out.size(0), -1)  # Flatten the output
        relu_out = self.relu(rnn_out)
        logits = self.linear(relu_out)

        return logits


In [10]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [11]:
hidden_layer_size = 64
sequence_length = 500

model = MBTIPredictor(hidden_layer_size, sequence_length).to(device)
print(model)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

MBTIPredictor(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

# Model Training

In [12]:
learning_rate = 3e-5
loss_fn = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=learning_rate)
batch_size = 20
epochs = 4

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

num_training_steps = epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="cosine", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [13]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    model.train()

    for batch, batch_data in enumerate(dataloader):

        # Compute prediction and loss
        input_ids = batch_data['input_ids'].to(device)
        attention_mask = batch_data['attention_mask'].to(device)
        labels = batch_data['labels'].to(device)

        pred = model(input_ids, attention_mask)
        loss = loss_fn(pred, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        lr_scheduler.step()

        if batch % 50 == 0:
            loss_value = loss.item()
            current = batch * len(input_ids)
            print(f"loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}]")

In [14]:
# Training loop
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer, device)

print("Training complete!")

Epoch 1
-------------------------------
loss: 2.777645  [    0/ 5789]
loss: 2.591379  [ 1000/ 5789]
loss: 2.693807  [ 2000/ 5789]
loss: 2.511338  [ 3000/ 5789]
loss: 2.676594  [ 4000/ 5789]
loss: 2.523948  [ 5000/ 5789]
Epoch 2
-------------------------------
loss: 2.393396  [    0/ 5789]
loss: 2.329754  [ 1000/ 5789]
loss: 2.450936  [ 2000/ 5789]
loss: 2.105190  [ 3000/ 5789]
loss: 2.186874  [ 4000/ 5789]
loss: 2.324069  [ 5000/ 5789]
Epoch 3
-------------------------------
loss: 2.417232  [    0/ 5789]
loss: 2.295762  [ 1000/ 5789]
loss: 2.103255  [ 2000/ 5789]
loss: 1.924069  [ 3000/ 5789]
loss: 2.082901  [ 4000/ 5789]
loss: 1.919485  [ 5000/ 5789]
Epoch 4
-------------------------------
loss: 2.085448  [    0/ 5789]
loss: 2.037217  [ 1000/ 5789]
loss: 1.898327  [ 2000/ 5789]
loss: 2.044947  [ 3000/ 5789]
loss: 2.232673  [ 4000/ 5789]
loss: 1.910356  [ 5000/ 5789]
Training complete!


In [15]:
def find_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch_data in dataloader:
            input_ids = batch_data['input_ids'].to(device)
            attention_mask = batch_data['attention_mask'].to(device)
            labels = batch_data['labels'].to(device)

            model_output = model(input_ids, attention_mask)

            _, predicted = torch.max(model_output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = round(100 * correct / total, 3)
    return accuracy

In [16]:
accuracy = find_accuracy(model, test_loader)
print(f'Model accuracy: {accuracy}%')

Model accuracy: 20.373%


# Saving Model State

In [17]:
model_save_path = '/content/final_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Model saved to /content/final_model.pth


In [18]:
files.download(model_save_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>